## Purpose
This script parses the pricing sheet for dry and frozen items and generated a unified xlsx with the price and other meta information like weight, number of pcs in case etc.

The inputs are the 2 pricing xlsx files from LB.
* PRICE LIST FOR DRY GOODS.xlsx
* PRICE LIST FOR FROZEN GOODS.xlsx

The output is a xlsx file with all the static and pricing information.
* lb_sku_pricing.xlsx

The general steps are
* Read the price list for dry goods.
* Create a data base of all the sku with the pricing and meta info.
* Read the price list for all frozen products.
* Create a data base of all sku with pricing and other info
* Add the 2 data base together.
* Write the data base to file *lb_sku_pricing.xlsx*

In [1]:
PRICING_DIR = "../private"
LB_DRY_PRICING = "PRICE LIST FOR DRY GOODS.xlsx"
LB_FROZEN_PRICING = "PRICE LIST FOR FROZEN GOODS.xlsx"

OUT_DIR = "out"
OUT_REPORT = "lb_sku_inventory.xlsx"

In [2]:
import re
import os
import openpyxl
from pathlib import Path

import sys
sys.path.append("../src")
from lb_pricing_parser import *

IN_DRY_XLSX = Path(PRICING_DIR, LB_DRY_PRICING)
IN_FROZEN_XLSX = Path(PRICING_DIR, LB_FROZEN_PRICING)
OUT_XLSX = Path(OUT_DIR, "lb_sku_pricing.xlsx")
    

if not os.path.exists(IN_DRY_XLSX):
    raise ValueError(f"Could not find {IN_DRY_XLSX} file")

if not os.path.exists(IN_FROZEN_XLSX):
    raise ValueError(f"Could not find {IN_FROZEN_XLSX} file")

if not os.path.exists(OUT_DIR):
    print("Creating output directory")
    os.mkdir(OUT_DIR)

print(f"Parsing {IN_DRY_XLSX} to create output file {OUT_XLSX}")

Parsing ../private/PRICE LIST FOR DRY GOODS.xlsx to create output file out/lb_sku_pricing.xlsx


In [3]:
wb = openpyxl.load_workbook(IN_DRY_XLSX)
print(wb)
sheet = wb.active

In [4]:
row_idx = extract_pricing_header_row_index(sheet)
print(row_idx)

Found header row at 9
9


In [5]:
col_idx_lst = extract_pricing_header_col_indices(sheet, row_idx)
print(col_idx_lst)

['SI.NO', 'SKU', 'IDENTIFICATION NO', 'DESCRIPTION OF GOODS', 'wt/pcs in gms', 'pcs/case', 'price/pcs', 'Price/case']
{'SKU': 2, 'Description': 4, 'wt_pc_in_gms': 5, 'pcs_per_case': 6, 'price_per_pc': 7}


In [6]:
dry_db = create_pricing_database(sheet, row_idx, col_idx_lst)

Found 82 entries in the database


In [7]:
wb = openpyxl.load_workbook(IN_FROZEN_XLSX)
print(wb)
sheet = wb.active

In [8]:
row_idx = extract_pricing_header_row_index(sheet)
print(row_idx)

Found header row at 12
12


In [9]:
col_idx_lst = extract_pricing_header_col_indices(sheet, row_idx)
print(col_idx_lst)

['Sl.NO', 'SKU', 'IDENTIFICATION NO', 'DESCRIPTION OF GOODS', 'wt/pcs in gms', 'pcs/case', 'price/pcs', 'price/case']
{'SKU': 2, 'Description': 4, 'wt_pc_in_gms': 5, 'pcs_per_case': 6, 'price_per_pc': 7}


In [10]:
cold_db = create_pricing_database(sheet, row_idx, col_idx_lst)


Found 21 entries in the database


In [11]:
db = dry_db + cold_db
print(f"Final data base created with {len(db)} entries")

Final data base created with 103 entries


In [12]:
save_pricing_database_to_xlsx(OUT_XLSX, db)

Saving output file 'out/lb_sku_pricing.xlsx'
